# Plot Maps of the Service Territory for the Balancing Authorities Analyzed


In [ ]:
# Start by importing the packages we need:
import os
import yaml

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable


## Set the Directory Structure


In [ ]:
# Identify the data and impage input and output directories:
ba_mapping_data_input_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/ba_service_territory/'
county_shapefile_input_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/county_shapefiles/'
state_shapefile_input_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/state_shapefiles/'
ba_to_process_input_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/'
image_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/figures/'


## Set the List of Balancing Authorities to Plot

BAs used in this analysis are controlled by a master file `balancing_authorities_modeled.yml` stored in the `/data` directory.

In [ ]:
# Read the yml file into a dictionary:
with open((ba_to_process_input_directory + 'balancing_authority_modeled.yml'), 'r') as yml:
     ba_list = yaml.load(yml, Loader=yaml.FullLoader)
     bas = [i for i in ba_list.keys()]

# Return the list of BAs to process/plot:
bas


## Create the Function to Plot the Service Territory for a Set of Balancing Authorities


In [ ]:
def plot_ba_service_territory(bas_to_plot: str, ba_mapping_data_input_directory: str, county_shapefile_input_directory: str, state_shapefile_input_directory: str,
                              image_output_directory: str, image_resolution: int, save_images=False):
    
    # Read in the county shapefile and reassign the 'FIPS' variable as integers:
    counties_df = gpd.read_file(os.path.join(county_shapefile_input_directory, r'tl_2020_us_county.shp')).rename(columns={'GEOID': 'County_FIPS'})
    counties_df['County_FIPS'] = counties_df['County_FIPS'].astype(int)

    # Read in the BA mapping file:
    ba_mapping_df = pd.read_csv((os.path.join(ba_mapping_data_input_directory, 'ba_service_territory_2019.csv')), index_col=None, header=0)

    # Merge the ba_mapping_df and counties_df together using county FIPS codes to join them:
    counties_df = counties_df.merge(ba_mapping_df, on='County_FIPS', how='left')

    # Subset to only the BA you want to plot:
    counties_subset_df = counties_df.loc[counties_df['BA_Code'].isin(bas_to_plot)]
    
    # Read in the state shapefile:
    states_df = gpd.read_file(os.path.join(state_shapefile_input_directory, 'tl_2020_us_state.shp')).rename(columns={'NAME': 'State'})
    
    # Create the figure:
    fig, ax = plt.subplots(1, 1, figsize=(25, 10))
    plt.rcParams['font.size'] = 18
    ax1 = counties_subset_df.plot(ax=ax, column='BA_Code', edgecolor='black', linewidth=1, legend=True, legend_kwds={'loc': 'lower right'})
    ax2 = states_df.boundary.plot(ax=ax, edgecolor='black', linewidth=1.5)
    ax.set_xlim(-126.4, -66);
    ax.set_xlabel('Longitude', fontsize=18); 
    ax.set_ylim(24, 50); 
    ax.set_ylabel('Latitude', fontsize=18);
    ax.set_title('Approximate Balancing Authority Service Territory in 2019', fontsize=21)

    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       plt.savefig(os.path.join(image_output_directory + 'BA_Service_Territory.png'), dpi=image_resolution, bbox_inches='tight')


## Make the Plot


In [ ]:
plot_ba_service_territory(bas_to_plot = bas,
                          ba_mapping_data_input_directory = ba_mapping_data_input_directory, 
                          county_shapefile_input_directory = county_shapefile_input_directory,
                          state_shapefile_input_directory = state_shapefile_input_directory,
                          image_output_directory = image_output_directory, 
                          image_resolution = 300, 
                          save_images = True)
